In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-17 20:20:17,228 | INFO: Optimizer: adam with learning_rate: 0.0001


In [4]:
model.skill.model.config

{'encoder': {'vocab_size': 30522,
  'hidden_size': 512,
  'intermediate_size': 2048,
  'num_hidden_layers': 3,
  'bidirectional': True},
 'decoder': {'intermediate_size': 2048, 'num_hidden_layers': 3}}

In [2]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
response_mask = dialog["response_mask_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
print(utterance, utterance.shape)
print(utterance_mask, utterance_mask.shape)
print(response, response.shape)
print(response_mask, response_mask.shape)
print(sentiment, sentiment.shape)

tensor([[  101,  2092,  1010,  ..., 23765,  1010,   102],
        [  101,  2192,  2033,  ...,     0,     0,     0],
        [  101, 14433,  1012,  ...,     0,     0,     0],
        ...,
        [  101,  2025,  2304,  ...,     0,     0,     0],
        [  101,  2025,  2091,  ...,  2157,  1012,   102],
        [  101,  2821,  1010,  ...,  1012,  1012,   102]], device='cuda:0') torch.Size([84, 20])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0') torch.Size([84, 20])
tensor([[  101,  2821,  1010,  ...,   102,     0,     0],
        [  101,  2182,  1012,  ...,  2869,  1029,   102],
        [  101,  3123,  1010,  ...,  9202,  9989,   102],
        ...,
        [  101,  2053,  1010,  ...,     0,     0,     0],
        [  101,  1045,  2031,  ..., 11007,  1997,   102],
        [  101,  3123,  1012,  ...,  2003

In [3]:
encoder_out = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(encoder_out.shape)
print(prev_output.shape)
print(target_output.shape)

torch.Size([84, 20, 4096]) torch.Size([6, 84, 2048]) torch.Size([6, 84, 2048])
torch.Size([84, 20, 4096]) torch.Size([3, 84, 4096]) torch.Size([3, 84, 4096])


RuntimeError: size mismatch, m1: [1680 x 4098], m2: [514 x 512] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:268

### for name, parameter in model.skill.model.named_parameters():
    print(name, parameter.requires_grad)

In [ ]:
for parameter in model.skill.model.decoder.fc3.parameters():
    print(parameter.requires_grad)

In [ ]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

In [ ]:
import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.lstm
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.lstm)
from nlptools.zoo.encoders.transformer import TransformerDecoder, TransformerEncoder
from nlptools.zoo.encoders.lstm import LSTMDecoder, LSTMEncoder

embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 3072
dropout = 0
decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask)
output.shape

In [ ]:
decoder